In [ ]:
import os
import pandas as pd
import kagglehub

# 1) Descarga y extracción
path = kagglehub.dataset_download("crowdflower/twitter-airline-sentiment")

# 2) Lee el archivo CSV descargado
df = pd.read_csv(os.path.join(path, "Tweets.csv"))

# 3) Muestra las primeras filas del DataFrame
print("Primeras 5 filas del DataFrame:\n", df.head())

ValueError: 'crowdflower/twitter-airline-sentiment' is not a valid KaggleDatasetAdapter